# Geo-mapping
We learned in lecture that there are several datasets that are publicly available about Covid-19 from the NY Times github site. Let's visualize this data in a geographic context.

In this assignment, the so-called automatic graders actually export your work into a gradeable format. They don't grade anything at all. In fact, the graders will choke if we embed maps into the Jupyter notebook, so we save the maps as HTML and run them from HTML files. The text "To test this, click here." will open the saved HTML file. 

First, let's re-run the example from the lecture:

In [1]:
# Run this to install required libraries folium and pyshp.
! pip install folium
! pip install pyshp

  Using cached folium-0.12.1.post1-py2.py3-none-any.whl (95 kB)
  Using cached branca-0.4.2-py3-none-any.whl (24 kB)
  Using cached pyshp-2.1.3.tar.gz (219 kB)
  Created wheel for pyshp: filename=pyshp-2.1.3-py3-none-any.whl size=37325 sha256=c5501090b96dff1d7e83ced654490211405b25a3c3c9d1c6a52c973c01aa75e1
  Stored in directory: /Users/shanneysuhendra/Library/Caches/pip/wheels/1f/1b/b5/54affbefc8a7e2bdf1da000fc576b8a1c91338f1f327a04f4c
Successfully built pyshp


In [2]:
# Load shapefile data from the US census site.
counties_url = "https://www2.census.gov/geo/tiger/GENZ2020/shp/cb_2020_us_county_20m.zip"
# cds_url = "https://www2.census.gov/geo/tiger/GENZ2020/shp/cb_2020_us_cd116_20m.zip"
# detailed_counties_url = 'https://www2.census.gov/geo/tiger/TIGER2021/COUNTY/tl_2021_us_county.zip'
counties_filename = 'counties.zip'
import requests
r = requests.get(counties_url)
# Load the file as a stream of 1024-byte chunks to avoid filling memory.
with open(counties_filename,'wb') as f:
  for chunk in r.iter_content(chunk_size=1024):
      f.write(chunk)

In [3]:
# For this to work, your computer has to have unzip available as a command. You might have to unzip it differently.
import zipfile
with zipfile.ZipFile(counties_filename, 'r') as zip_ref:
    zip_ref.extractall()

In [4]:
# Construct a GeoJSON equivalent for an existing ShapeFile.
# This is necessary to get past the examples for folium. 
import shapefile
def shapefile2geojson(filename):
    reader = shapefile.Reader(filename)
    # read the shapefile and output compatible GeoJson
    # What fields are in the shapefile?
    fields = reader.fields[1:]
    # what are their names?
    field_names = [field[0] for field in fields]
    buffer = []
    for sr in reader.shapeRecords():
        # Build a GeoJSON record corresponding to the Shape record.
        atr = dict(zip(field_names, sr.record))        
        geom = sr.shape.__geo_interface__
        buffer.append(dict(type="Feature", geometry=geom, properties=atr))
    # Return the overall json structure as a python dictionary
    return {'type': 'FeatureCollection', 'features': buffer}

In [5]:
shapes = shapefile2geojson('cb_2020_us_county_20m.shp')

In [6]:
# read and massage some NYT data
import pandas as pd
geodata = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties-recent.csv')
geodata = geodata[geodata['fips'].notna()]  # some data has no fips code. Don't include those rows. 
geodata = geodata[geodata['date'] == max(geodata['date'])]   # get data for the latest date. This updates daily. 
geodata['GEOID'] = geodata['fips'].map(lambda x: "{:05d}".format(int(x)))  # print the appropriate GEOID in text format. 
geodata

,date,county,state,fips,cases,deaths,GEOID
94263,2021-12-13,Autauga,Alabama,1001.0,10605,158.0,01001
94264,2021-12-13,Baldwin,Alabama,1003.0,38321,591.0,01003
94265,2021-12-13,Barbour,Alabama,1005.0,3716,81.0,01005
94266,2021-12-13,Bibb,Alabama,1007.0,4379,95.0,01007
94267,2021-12-13,Blount,Alabama,1009.0,10864,193.0,01009
...,...,...,...,...,...,...,...
97509,2021-12-13,Sweetwater,Wyoming,56037.0,8204,103.0,56037
97510,2021-12-13,Teton,Wyoming,56039.0,5446,14.0,56039
97511,2021-12-13,Uinta,Wyoming,56041.0,4094,31.0,56041
97512,2021-12-13,Washakie,Wyoming,56043.0,1852,36.0,56043


In [7]:
import folium
m = folium.Map(location=[48, -102], zoom_start=3)
folium.Choropleth(
    geo_data=shapes,
    name="choropleth",
    data=geodata,
    columns=["GEOID", "cases"],  # where to find the key and value in geodata
    key_on="feature.properties.GEOID",  # path to the feature ID in geojson
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Covid Cases",
).add_to(m)

folium.LayerControl().add_to(m)
m.save('hw10_m_map.html')
# do not display a map; this will cause your notebook not to save properly!

To test this, [Click here](hw10_m_map.html).

The whole point of this homework is to utilize Ny Times Covid data and US census data instead of the canned dataset in the demonstration github site. 

*Question 1:* According to this map, considering only the cases, the data for Los Angeles County makes it the worst possible place to be. Renormalize this data by converting it to percentages of population by downloading the latest census data and dividing each county's cases by its population. Plot the result using the intensity of red as the depiction of a percentage. *Hints:*
* You can use the existing shapefiles already read into `shapes`.
* You'll want the most recent county population data (the last column) from the following dataset found at "https://www2.census.gov/programs-surveys/popest/datasets/2010-2020/counties/totals/co-est2020.csv". I'll read it for you below to keep you away from the frustration I experienced. It has a few malformed lines and `pd.read_csv` doesn't work. I used the python `csv` library instead. 
* The population `DataFrame` is `pop`.
* The county populations are in the column `pop['POPESTIMATE2020']`.
* You might have to convert strings to numbers. 
* Columns `STATE` and `COUNTY` can be concatenated to produce the FIPS code. This FIPS code matches the GEOID in the `shapes` JSON file. 
* To combine data from different `DataFrame`s, *they must have the same index.* Use `pd.DataFrame.set_index(column_name)` to accomplish this. 
* When you're done combining, `pd.DataFrame.reset_index()` will recover the column that was the index. 
* Both `set_index` and `reset_index` return the transformed `DataFrame` and leave their arguments unchanged. You need to use the return value. 

In [8]:
import csv
import requests
def df_from_csv_url(url): 
    data = requests.get(url)
    lines = data.text.split('\n')
    rows = []
    for row in csv.reader(lines):
        rows.append(row)
    labels = rows[0]
    columns = []
    for c in range(len(labels)):
        column = []
        for r in range(1, len(rows)):
            row = rows[r]
            if len(row) == len(labels): 
                column.append(row[c])
        columns.append(column)
    
    js = {}
    for c in range(len(columns)):
        js[labels[c]] = columns[c]

    return pd.DataFrame(js)

In [9]:
pop_url = "https://www2.census.gov/programs-surveys/popest/datasets/2010-2020/counties/totals/co-est2020.csv"
pop = df_from_csv_url(pop_url)
pop

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,POPESTIMATE2016,POPESTIMATE2017,POPESTIMATE2018,POPESTIMATE2019,POPESTIMATE042020,POPESTIMATE2020
0,040,3,6,01,000,Alabama,Alabama,4779736,4780118,4785514,...,4816632,4831586,4843737,4854803,4866824,4877989,4891628,4907965,4920706,4921532
1,050,3,6,01,001,Alabama,Autauga County,54571,54582,54761,...,54970,54747,54922,54903,55302,55448,55533,55769,56130,56145
2,050,3,6,01,003,Alabama,Baldwin County,182265,182263,183121,...,190203,194978,199306,203101,207787,212737,218071,223565,227989,229287
3,050,3,6,01,005,Alabama,Barbour County,27457,27454,27325,...,27172,26946,26768,26300,25828,25169,24887,24657,24652,24589
4,050,3,6,01,007,Alabama,Bibb County,22915,22904,22858,...,22657,22510,22541,22553,22590,22532,22300,22313,22199,22136
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3189,050,4,8,56,037,Wyoming,Sweetwater County,43806,43806,43580,...,45032,45189,44996,44780,44319,43663,43188,42917,42717,42673
3190,050,4,8,56,039,Wyoming,Teton County,21294,21298,21298,...,21643,22335,22801,23083,23255,23383,23261,23385,23453,23497
3191,050,4,8,56,041,Wyoming,Uinta County,21118,21121,21090,...,21008,20969,20835,20777,20711,20449,20299,20196,20169,20215
3192,050,4,8,56,043,Wyoming,Washakie County,8533,8528,8531,...,8410,8417,8277,8282,8180,8013,7886,7824,7756,7760


In [10]:
# Please put your folium map into the variable "q1". 
# BEGIN SOLUTION
pop['GEOID'] = pop['STATE'] + pop['COUNTY']
new_d = pop.merge(geodata, on='GEOID')
new_d['POPESTIMATE2020'] = new_d['POPESTIMATE2020'].astype('int32')
new_d['COVIDPERCENT'] = (new_d['cases']/new_d['POPESTIMATE2020'])*100

q1 = folium.Map(location=[48, -102], zoom_start=3)
folium.Choropleth(
    data=new_d,
    fill_color="OrRd",
    fill_opacity=0.7,
    line_opacity=0.2,
    geo_data=shapes,
    name="choropleth",
    columns=["GEOID", "COVIDPERCENT"], 
    key_on="feature.properties.GEOID",  
    legend_name="percentages % of Covid Cases",
).add_to(q1)
folium.LayerControl().add_to(q1)
# END SOLUTION
q1.save('hw10_q1_map.html')  # save the map
# do not display a map; this will cause your notebook not to save properly!

To test this, [click here](hw10_q1_map.html).

*Question 2:* Based upon this new visualization, where are the real worst places to live if you don't want to catch Covid? 

In [11]:
# Demonstrate your answer here. 
# BEGIN SOLUTION
print(new_d.sort_values(by='COVIDPERCENT', axis=0, ascending=False).head(5)[['STNAME', 'CTYNAME', 'COVIDPERCENT']])
print(new_d.sort_values(by='COVIDPERCENT', axis=0, ascending=False).head(50)[['STNAME', 'COVIDPERCENT']])
# END SOLUTION

            STNAME               CTYNAME  COVIDPERCENT
407        Georgia  Chattahoochee County     58.629514
252       Colorado        Crowley County     43.644663
2576         Texas         Dimmit County     38.528967
70          Alaska    Bethel Census Area     37.299995
2371  South Dakota          Dewey County     37.208499
            STNAME  COVIDPERCENT
407        Georgia     58.629514
252       Colorado     43.644663
2576         Texas     38.528967
70          Alaska     37.299995
2371  South Dakota     37.208499
244       Colorado     35.922330
82          Alaska     31.213686
83          Alaska     30.886970
951         Kansas     30.555556
1273      Michigan     29.660464
2465     Tennessee     29.164282
145       Arkansas     29.055933
2502     Tennessee     28.529027
79          Alaska     28.170029
139       Arkansas     27.915364
2024  North Dakota     27.816364
2640         Texas     27.567151
1013      Kentucky     26.773980
198     California     26.749067
2918      

The worst counties to live in by ranking (1. being the worst):
1- Chattahoochee County in Georgia
2- Crowley COunty in Colorado, 
3- Dimmit County in Texas, 
4- Dewey Conty in South Dakota
5- Bethel Census Area in Alaska.
additionally, the state alaska has 3 counties on the top 10 worst counties to live in.

*Question 3:* Let's investigate short term trends. Load a new dataset from the same source as `geodata` but make it correspond to the earliest date in the short-term data. Compute differences between latest and earliest data, and plot in a choropleth. The same hints about combining datasets apply.  

In [12]:
# Please put your folium map into the variable "q3".
# BEGIN SOLUTION
new_geod = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties-recent.csv')
new_geod = new_geod[new_geod['fips'].notna()] 
#get data for earliest date
new_geod = new_geod[new_geod['date'] == min(new_geod['date'])]
#print the appropriate GEOID in text format
new_geod['GEOID'] = new_geod['fips'].map(lambda x: "{:05d}".format(int(x)))  

#merges dataset
new_geod1 = new_geod.merge(geodata, on = 'GEOID', suffixes = ['_OLD', '_NEW'])
new_geod1['change'] = new_geod1['cases_NEW']-new_geod1['cases_OLD']


q3 = folium.Map(location=[48, -102], zoom_start=3)
folium.Choropleth(
    data=new_geod1,
    fill_color="OrRd",
    fill_opacity=0.7,
    line_opacity=0.2,
    geo_data=shapes,
    name="choropleth",
    columns=["GEOID", "change"],
    key_on="feature.properties.GEOID",
    legend_name="Change in quantity of Covid cases (April-November)",
).add_to(q3)
folium.LayerControl().add_to(q3)
# END SOLUTION 
q3.save("hw10_q3_map.html")
# do not display a map; this will cause your notebook not to save properly!

To test this, [click here](hw10_q3_map.html).

*Question 4:* Based upon this new visualization, where are the real worst places to live *right now* if you don't want to catch Covid? 

In [13]:
# Enter any exploratory code here to prove your point. 
# BEGIN SOLUTION
print(new_geod1.sort_values(by='change', axis=0, ascending=False).head(10)[['state_OLD', 'county_OLD', 'change']])
# END SOLUTION

         state_OLD   county_OLD  change
101        Arizona     Maricopa   62457
608       Illinois         Cook   51962
202     California  Los Angeles   36833
1310      Michigan        Wayne   36238
1291      Michigan      Oakland   26859
2055          Ohio     Cuyahoga   22813
1278      Michigan       Macomb   22590
1338     Minnesota     Hennepin   21668
1872      New York      Suffolk   21583
2240  Pennsylvania    Allegheny   19127


The counties whose cases increased the most from April to November are (1 being the most): 1-Maricope, 2-Cook, 3-LA, 4-Wayne, 5-Oakland.  
additionally michigan has 3 counties that is in the top 10 of most increased cases, with 2 of those counties in the top 5. based on this data alone it would seem that living in michigan would not be a good idea. however, it is important to note that this data is the number of cases, not the percentage of the population. thus, the data is not necessarilly complete for us to determine which county/state is the worst place to live in.

In [14]:
# To submit your work, SAVE YOUR NOTEBOOK and then run this cell, and then upload the result to GradeScope.
! python -m zipfile -c hw10.zip hw10.ipynb hw10_q1_map.html hw10_q3_map.html
from IPython.display import display, HTML, FileLink
display(HTML("Please download the link"), FileLink('hw10.zip'), HTML("and upload to GradeScope."))

/Users/shanneysuhendra/Desktop/comp30/hws/hw10/hw10.zip